In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image
from MM_Difussion import combinedCLIP
from shap_e.models.generation.latent_diffusion import SplitVectorDiffusion

/mnt/c/Users/Robinson/OneDrive/Desktop/Classes_Fall_2024/CAP6411/Project_Real/shap-e/shap-e/shap_e/models/nn/checkpoint.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length, *args):
/mnt/c/Users/Robinson/OneDrive/Desktop/Classes_Fall_2024/CAP6411/Project_Real/shap-e/shap-e/shap_e/models/nn/checkpoint.py:44: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *output_grads):
/mnt/c/Users/Robinson/OneDrive/Desktop/Classes_Fall_2024/CAP6411/Project_Real/shap-e/shap-e/shap_e/models/nn/checkpoint.py:62: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length_1, length_2, *args):
/mnt/c/Users/Robinson/OneDrive/Desktop/Classes_Fall_2024

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
model_text = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

/mnt/c/Users/Robinson/OneDrive/Desktop/Classes_Fall_2024/CAP6411/Project_Real/shap-e/shap-e/shap_e/models/download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retu

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

/mnt/c/Users/Robinson/OneDrive/Desktop/Classes_Fall_2024/CAP6411/Project_Real/shap-e/shap-e/shap_e/models/download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retu

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [4]:
cclip = combinedCLIP(device=device, dtype=torch.float32, n_ctx=1024, token_cond=True, frozen_clip=True, width=1024, input_channels=1024, output_channels=2048, time_token_cond=True, layers=24, use_pos_emb=True).to(device)

1024


In [5]:
finalcclip = SplitVectorDiffusion(device=device, wrapped=cclip, n_ctx=1024, d_latent=1024*1024)

In [6]:

def load_state_dict(model, model_image, model_text, mode="image"):
    j = 0
    for i, k in model.named_parameters():
        if i in model_text.state_dict().keys() and i in model_image.state_dict().keys():
            if mode == "average":
                k.data = (model_image.state_dict()[i] + model_text.state_dict()[i])/2
            elif mode == "text":
                k.data = model_text.state_dict()[i]
            elif mode == "image":
                k.data = model_image.state_dict()[i]
            else:
                raise ValueError("Mode not recognized")
            j+=1
        else:
            if i in model_text.state_dict().keys():
                k.data = model_text.state_dict()[i]
                j+=1
            else:
                key = i.replace("_img", "")
                k.data = model_image.state_dict()[key]
                j+=1
        
        print(j)

In [7]:
len(finalcclip.state_dict().keys())

307

In [8]:
load_state_dict(finalcclip, model, model_text)

Copying wrapped.pos_emb
1
Copying wrapped.time_embed.c_fc.weight
2
Copying wrapped.time_embed.c_fc.bias
3
Copying wrapped.time_embed.c_proj.weight
4
Copying wrapped.time_embed.c_proj.bias
5
Copying wrapped.ln_pre.weight
6
Copying wrapped.ln_pre.bias
7
Copying wrapped.backbone.resblocks.0.attn.c_qkv.weight
8
Copying wrapped.backbone.resblocks.0.attn.c_qkv.bias
9
Copying wrapped.backbone.resblocks.0.attn.c_proj.weight
10
Copying wrapped.backbone.resblocks.0.attn.c_proj.bias
11
Copying wrapped.backbone.resblocks.0.ln_1.weight
12
Copying wrapped.backbone.resblocks.0.ln_1.bias
13
Copying wrapped.backbone.resblocks.0.mlp.c_fc.weight
14
Copying wrapped.backbone.resblocks.0.mlp.c_fc.bias
15
Copying wrapped.backbone.resblocks.0.mlp.c_proj.weight
16
Copying wrapped.backbone.resblocks.0.mlp.c_proj.bias
17
Copying wrapped.backbone.resblocks.0.ln_2.weight
18
Copying wrapped.backbone.resblocks.0.ln_2.bias
19
Copying wrapped.backbone.resblocks.1.attn.c_qkv.weight
20
Copying wrapped.backbone.resblocks

In [10]:


batch_size = 4
guidance_scale = 3.0
prompt = "a corgi in a field"

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("example_data/corgi.png")

latents = sample_latents(
    batch_size=batch_size,
    model=finalcclip,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size, texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=5,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/5 [00:00<?, ?it/s]

2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])


In [11]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAHN9enN8eXJ8eXJ7eXF7eHF6d3B6d3B5d3B5dm95d295dnF4d3…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAHN9enN8enN8eXJ8eXJ7eXF7eHF6eHF6d3B6d3B5d3B5dnF4d2…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAHN9enN8eXJ8eXJ7eXJ7eHF6eHF6d3B6d3B5d3B5dm95d3F4dm…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAHN9enN8enN8eXJ8eXF7eHF6eHF6d3B6d3B5d3B5dm95d3J4d3…

In [12]:
latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=5,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/5 [00:00<?, ?it/s]

2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])
2048
torch.Size([8, 2048, 1024])


In [20]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOTh4OPg3+bf3+Pe3OLd1+He3d3d2OHc3Nzb2eTXy9/Yz93W0d…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPj28vXz8PXz7/Tz7/Py7vPy7fPx7fbw8ffu7fXs6u3s6uzs6O…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPnr5/Tm4u7h3+3g3+zg3uzf2uje2+3d3Ond3Onb2ubc2u3Y0+…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPXz6+7t6uvq6O3o5enp5e7n4Ofl4+Tk4OPj3eXi2uDh29/b1d…